In [1]:
import numpy as np
import matplotlib.pyplot as pl
#%matplotlib qt5
from tool._fixedInt import *

In [2]:
def rcosine(beta, Tbaud, oversampling, Nbauds, Norm):
    """ Respuesta al impulso del pulso de caida cosenoidal """
    t_vect = np.arange(-0.5*Nbauds*Tbaud, 0.5*Nbauds*Tbaud, 
                       float(Tbaud)/oversampling) #limitacion de la respuesta al impulso

    y_vect = [] 
    #generacion del conseno realsado
    for t in t_vect:
        y_vect.append(np.sinc(t/Tbaud)*(np.cos(np.pi*beta*t/Tbaud)/    
                                        (1-(4.0*beta*beta*t*t/
                                            (Tbaud*Tbaud)))))

    y_vect = np.array(y_vect)

    if(Norm):
        return (t_vect, y_vect/np.sqrt(np.sum(y_vect**2)))
        #return (t_vect, y_vect/y_vect.sum())
    else:
        return (t_vect,y_vect)

In [3]:
## Parametros generales
Baud  = 1.0e9           # baudrate 
T     = (1.0/Baud)     # Periodo de baudio
Nsymb = 100             # Numero de simbolos
os    = 4               # Factor de sobre muestreo(oversampling), cantidad de muestras por periodo para representar el coseno 

## Parametros de la respuesta en frecuencia
Nfreqs = 256            # Cantidad de frecuencias

## Parametros del filtro de caida cosenoidal
beta   = [0.0,0.5,1.0]  # Roll-Off
Nbauds = 6.0           # Cantidad de baudios del filtro(largo de la respuesta al impulso)
## Parametros funcionales
Ts = T/os               # Frecuencia de muestreo 

In [4]:
(t,h) = rcosine(beta[1], T,os,Nbauds,Norm=False)

In [8]:
#cuantificacion #S(8,7) round
NB=8
NBF=7
Mode='round'
h_PF= arrayFixedInt(NB,NBF,h,signedMode='S',roundMode='round',saturateMode='saturate')
h_intValue = np.zeros((int(os),int(Nbauds))) ##valores a para el filtro en verilog
h_fValue   = np.zeros((int(os),int(Nbauds)))   ##valores para trabajar en python
a=[] #coeficientes
b=[]
for i in range(len(h_PF)):
    a.append(h_PF[i].intvalue)
    b.append(h_PF[i].fValue)
for i in range(int(os)):
    h_intValue[i]=a[i::int(os)]
    h_fValue[i]=b[i::int(os)]
print(h_intValue)
print(h_fValue)
x=np.array([0,0,0,0,0])
#print(h_PF[0].showValueRange())
# print(bin(129))
# print(bin(127))

[[  0.   0.   0. 127.   0.   0.]
 [  1. 249.  34. 114. 240.   3.]
 [  2. 241.  77.  77. 241.   2.]
 [  3. 240. 114.  34. 249.   1.]]
[[ 0.         0.         0.         0.9921875  0.         0.       ]
 [ 0.0078125 -0.0546875  0.265625   0.890625  -0.125      0.0234375]
 [ 0.015625  -0.1171875  0.6015625  0.6015625 -0.1171875  0.015625 ]
 [ 0.0234375 -0.125      0.890625   0.265625  -0.0546875  0.0078125]]


In [9]:
#-------------parametros PRBS
PRBS_I=np.array([1,1,0,1,0,1,0,1,0]) #0x1AA Semilla
PRBS_Q=np.array([1,1,1,1,1,1,1,1,0]) #0x1FE semilla
Nsym=((2**len(PRBS_I))-1)**2 + 1000 #cantidad de simbolos a transmitir, necesito bastante simbolos por que en el checker necesito contar 511
s=[]                                #convinaciones posibles por 511 valores mas los simboles que transmito para contar BER
#-------------parametros fir polifasico
output_fir_I=np.zeros(int(os)) #para las 4 salidas del fir polifasico
output_fir_Q=np.zeros(int(os))
reg_I=np.zeros(len(h_fValue[0])-1) #registro para el filtro fir
reg_Q=np.zeros(len(h_fValue[0])-1)
output_I=[]         #salida del filtro
output_Q=[]
output_conv=[]      #para corroborar con la salida del filtro
i_enb_tx=[]
#-------------parametros de PRBS checker
Phase=0 #por cada fase elejida se resetar el checker y busca otra vez la latencia
PRBS_errors_min=511
PRBS_index_min=0
prbs_errors_count=0
PRBS_pos=0 # para ir cambiando la posicion cuando el erro no es cero, selector del mux en verilog
filter_delay=(len(a)/2/os) #lantencia de 12 pero al estar dividido en 4 fases seria de 3, en rtl hacer un reset mas tarde.
PRBS_checker_reg_I = np.zeros(int(2**len(PRBS_I))-1) #resgistro para ir guardando los simbolos entrante
PRBS_checker_reg_Q = np.zeros(int(2**len(PRBS_Q))-1)
PRBS_checker_count =0 #para llevar la cuenta y comprar con los 511 valores de la PRBS 
PRBS_checker_locked =False #para decir que recorri 511 valores en 511 posiciones posibles y obtener la latencia
errors_count_I=0
errors_count_Q=0
bits_count_I =0
bits_count_Q =0
#-------------PRBS----------
# descomentar hace falta la alinear la latencia del verilog con la salida de python
for a in range(50):             #espero 50 unidad de tiempo para luego habilitar el filtro junto con la PRBS
    i_enb_tx.append(0)
    output_I.append(0)
    output_Q.append(0)
for r in range(6):
    output_I.append(0)
    output_Q.append(0)
for i in range(Nsym):
    i_enb_tx.append(1)          #habilito la PRBS y el filtro en verilog DESCOMENTAR!!!!!
    aux= PRBS_I[0]^PRBS_I[4]
    PRBS_I = np.roll(PRBS_I,-1)
    PRBS_I[8] = aux
    #PRBS_I[0] simbolo que se transmite
    #s.append(PRBS_I[0]) #para verificacion del filtro FIR
    #s.extend([0]*3)  
    aux1= PRBS_Q[0]^PRBS_Q[4]
    PRBS_Q = np.roll(PRBS_Q,-1)
    PRBS_Q[8] = aux1
    #PRBS_Q[0] simbolo que se transmite
    #convierto para poder realizar la convolucion
    sim_I =  1 if PRBS_I[0]==0 else -1
    sim_Q =  1 if PRBS_Q[0]==0 else -1     
#------------filtro poli--------------
    for e in range(int(os)):
        output_fir_I[e]=sim_I*h_fValue[e][0]+np.dot(reg_I,h_fValue[e][1:]) 
        output_I.append(output_fir_I[e]) #salida del filtro FIR
        output_fir_Q[e]=sim_Q*h_fValue[e][0]+np.dot(reg_Q,h_fValue[e][1:])
        output_Q.append(output_fir_Q[e]) #salida del filtro FIR
    reg_I=np.roll(reg_I,1)
    reg_Q=np.roll(reg_Q,1)
    reg_I[0] = sim_I
    reg_Q[0] = sim_Q
#------------ dowslampling y slicer para el PRBS_checker-----------
    bit_I = 1 if output_fir_I[Phase] <=0 else 0
    bit_Q = 1 if output_fir_Q[Phase] <=0 else 0
#----------PRBS_checker-------------
    if i >= filter_delay: #descarto las muestras que no son validad
        #hago un shiftreg con la PRBS para ir comparando con los bit trasmitido y encontrar la latencia
        PRBS_checker_reg_I = np.roll(PRBS_checker_reg_I,1) #resgistro de desplazamiento
        PRBS_checker_reg_I[0] = PRBS_I[0]
        PRBS_checker_reg_Q = np.roll(PRBS_checker_reg_Q,1) #resgistro de desplazamiento
        PRBS_checker_reg_Q[0] = PRBS_Q[0]

        if not PRBS_checker_locked:
            prbs_errors_count += (bit_I ^ int(PRBS_checker_reg_I[PRBS_pos])) #cuenta los errores
            if PRBS_checker_count == (2**(len(PRBS_I))-2):  #cuento hasta 510 antes que se vuelva a repetir la secuencia de la PRBS
                PRBS_checker_count = 0
                if (prbs_errors_count < PRBS_errors_min):  #compruebo si el error contado es menor a un error min y guardo
                    PRBS_errors_min = prbs_errors_count
                    PRBS_index_min = PRBS_pos
                if (PRBS_pos == (2**len(PRBS_I))-2):#cuando ya recorri 511 valores en 511 posiciones posibles bloqueo el checker
                    PRBS_checker_locked = True
                    print('checker locked in index =',PRBS_index_min)
                    #print(i)
                    #print(PRBS_pos)
                else:
                    PRBS_pos +=1
                prbs_errors_count=0
            else:
                PRBS_checker_count += 1
        else:
            errors_count_I += (bit_I ^ int(PRBS_checker_reg_I[PRBS_index_min])) #para contar errores con la latencia ya fija
            errors_count_Q += (bit_Q ^ int(PRBS_checker_reg_Q[PRBS_index_min])) 
            bits_count_I +=1
            bits_count_Q +=1
    
print('error de bits canal I:',errors_count_I)
print('cantidad de bits enviados canal I:',bits_count_I)

#output_conv=np.convolve(h,d)
# pl.figure(figsize=[14,7])
# pl.subplot(2,1,1)
# pl.plot(output_I,'r')
# #pl.plot(output_conv)
# pl.grid(True)
# pl.legend()
# pl.xlim(0,1000)

# pl.subplot(2,1,2)
# #pl.stem(s,use_line_collection=True)
# pl.plot(output_Q,'r')
# pl.grid(True)
# pl.legend()
# pl.xlim(0,1000)

checker locked in index = 3
error de bits canal I: 0
cantidad de bits enviados canal I: 997


In [18]:
o_pf_I= arrayFixedInt(11,7,output_I,signedMode='S',roundMode='round',saturateMode='saturate')
o_pf_I_v=[]
o_pf_Q= arrayFixedInt(11,7,output_Q,signedMode='S',roundMode='round',saturateMode='saturate')
o_pf_Q_v=[]
for i in range(len(o_pf_I)):
    o_pf_I_v.append(o_pf_I[i].intvalue)
    o_pf_Q_v.append(o_pf_Q[i].intvalue)
file0 = open('./filtro/fir_I.out','w')
file1 = open('./filtro/fir_Q.out','w')

for e in range(len(o_pf_I_v)):
    file0.write('%d\n'%o_pf_I_v[e])
    file1.write('%d\n'%o_pf_Q_v[e])
file0.close()
file1.close()

file2 = open('./filtro/i_enb_tx.out','w')
for q in range(len(i_enb_tx)):
    file2.write('%d\n'%i_enb_tx[q])
file2.close()

In [23]:
file2_copia = open('./filtro/i_enb_tx.out','r')
i_enb_tx_copia = file2_copia.read()
print(len(i_enb_tx))
print(len(i_enb_tx_copia))
file2_copia.close()

262171
524342


In [22]:
print(len(i_enb_tx))

262171
